# Copyright Netherlands eScience Center <br>
** Function     : Interpolation verification via iris** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2020.07.23 ** <br>
** Last Update  : 2020.07.23 ** <br>
Description     : This notebook aims to verify the interpolation of NSIDC sea ice data onto the ERA-Interim grid.<br>
Return Values   : netCDF4 <br>
Caveat          : The data is further processed by certain algorithms from NASA Goddard Center. The dataset consists of mulitple variables, but we only use the Merged GSFC NASA Team/Bootstrap daily sea ice concentrations from 1978 through most recent processing (variable name:"goddard_merged_seaice_conc"). It combines the results from two algorithms by NASA The details about these two algorithms are provided via the link:<br>
https://nsidc.org/support/faq/nasa-team-vs-bootstrap-algorithm<br>

More information (incl. description of variables) is available through:<br>
https://nsidc.org/data/g02202<br>

The projection center coordinate (refernce for ygrid and xgrid) info is included in the netCDF file:<br>
Latitude: North pole (90 deg)<br>
Longitude: -45 deg w.r.t. 0 deg (greenwitch)<br>

The coordinate values (latitude & longitude) are the exact values.<br>

We also apply nearest neighbour interpolation to the ERA-Interim grid. We use the iris module.<br>

Reference
Meier, W. N., F. Fetterer, M. Savoie, S. Mallory, R. Duerr, and J. Stroeve. 2017. NOAA/NSIDC Climate Data Record of Passive Microwave Sea Ice Concentration, Version 3. Boulder, Colorado USA. NSIDC: National Snow and Ice Data Center. doi: https://doi.org/10.7265/N59P2ZTG.<br>
Peng, G., W. N. Meier, D. Scott, and M. Savoie. 2013. A long-term and reproducible passive microwave sea ice concentration data record for climate studies and monitoring, Earth Syst. Sci. Data. 5. 311-318. https://doi.org/10.5194/essd-5-311-2013<br>

In [1]:
import numpy as np
import scipy
from netCDF4 import Dataset
import os
import glob
import iris
import iris.plot as iplt
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
################################   Input zone  ######################################
# specify data path
# SICpm fields
datapath = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/SIC_passive_microwave_NASA/daily'
datapath_coordinate = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ERA-Interim'
# sample
benchmark_key = Dataset(os.path.join(datapath,"2004","seaice_conc_daily_nh_f13_20040112_v03r01.nc"))
coordinate_key = Dataset(os.path.join(datapath_coordinate,"sic_weekly_erai_1979_2017.nc"))
# specify output path for figures
output_path = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/SIC_passive_microwave_NASA'
####################################################################################

In [3]:
#########################   Basic dimensions of NSIDC sic  #########################
latitude = benchmark_key.variables['latitude'][:]
longitude = benchmark_key.variables['longitude'][:]
ygrid = benchmark_key.variables['ygrid'][:]
xgrid = benchmark_key.variables['xgrid'][:]
sic_sample = benchmark_key.variables['goddard_merged_seaice_conc'][0,:,:]
print(sic_sample.shape)
print(latitude.shape)
print(longitude.shape)
print(ygrid.shape)
print(xgrid.shape)
print(np.amax(latitude))
print(np.amin(latitude))
print(np.amax(longitude))
print(np.amin(longitude))
print(np.amax(sic_sample))
print(np.amin(sic_sample))
####################################################################################

(448, 304)
(448, 304)
(448, 304)
(448,)
(304,)
89.8368159996152
31.1026717524309
179.813975395493
-180.0
1.0
0.0


In [4]:
#######################   Target coordinate for interpolation   ######################
latitude_ERAI = coordinate_key.variables['latitude'][:]
longitude_ERAI = coordinate_key.variables['longitude'][:]
#value_sample_coordinate = coordinate_key.variables['sic'][0,0,:,:]
#print(lat_coordinate)
#print(lon_coordinate)
#print(value_sample_coordinate[:3,:])
#reshape_test = np.reshape(value_sample_coordinate,[len(lat_coordinate)*len(lon_coordinate)], order='F')
#print(reshape_test[:100])
#reshape_back = np.reshape(reshape_test,[len(lat_coordinate),len(lon_coordinate)], order='F')
#print(reshape_back[:3,:])
######################################################################################

In [13]:
# basic dimensions for cube in iris
lat_iris = iris.coords.AuxCoord(latitude, standard_name='latitude', long_name='latitude',
                                            var_name='lat', units='degrees')
lon_iris = iris.coords.AuxCoord(longitude, standard_name='longitude', long_name='longitude',
                                            var_name='lon', units='degrees')
# assembly the cube
cube_iris = iris.cube.Cube(sic_sample, long_name='unstructured field', var_name='field', 
                            units='1', aux_coords_and_dims=[(lat_iris, (0,1)), (lon_iris, (0,1))])
coord_sys = iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS)
cube_iris.coord('latitude').coord_system = coord_sys
cube_iris.coord('longitude').coord_system = coord_sys
projection = ccrs.PlateCarree()
lat_grid = latitude_ERAI
lon_grid = longitude_ERAI
lat_aux = iris.coords.DimCoord(lat_grid, standard_name='latitude',
                                               units='degrees_north', coord_system='GeogCS')
lon_aux = iris.coords.DimCoord(lon_grid, standard_name='longitude',
                                               units='degrees_east', coord_system='GeogCS')
dummy_data = np.zeros((len(lat_grid), len(lon_grid)))
cube_tar = iris.cube.Cube(dummy_data,dim_coords_and_dims=[(lat_aux, 0), (lon_aux, 1)])
# create the coordinate system for the target cube
cube_tar.coord('latitude').guess_bounds()
cube_tar.coord('longitude').guess_bounds()
cube_tar.coord('latitude').coord_system = coord_sys
cube_tar.coord('longitude').coord_system = coord_sys
# create a weight matrix for regridding
weights = np.ones(cube_iris.shape)
# get regridder from given cubes
base = iris.analysis.UnstructuredNearest()
regridder = base.regridder(cube_iris,cube_tar)
# Transform cube to target projection
cube_regrid = regridder(cube_iris)
regrid_sic = cube_regrid.data
regrid_sic[regrid_sic>1.5] = 0

In [14]:
print(regrid_sic[20,265])
print(latitude_ERAI[20])
print(longitude_ERAI[264])
print(np.where(sic_sample==regrid_sic[20,265]))
print(latitude[np.where(sic_sample==regrid_sic[20,265])])
print(longitude[np.where(sic_sample==regrid_sic[20,265])])

0.3700000047683716
74.5
18.0
(array([ 27,  37,  60,  81,  94,  95, 100, 100, 104, 113, 115, 158, 158,
       161, 173, 197, 227, 228, 231, 233, 242, 263, 263, 263, 274, 283,
       285, 288, 288, 295, 315, 321, 340, 341, 414, 417]), array([117, 124, 114, 125, 120, 122, 114, 119, 106,  96,  87,  41,  53,
        67,  25,  79, 222, 222, 248, 219, 227, 186, 193, 214, 257, 278,
       288, 274, 275, 189, 138, 189, 171, 170,  95, 107]))
[44.12550116 46.30687293 50.51334019 55.25919654 57.73727502 58.04273844
 58.64596406 58.93260082 58.92169753 59.87338303 59.36302906 59.44871057
 61.56602616 64.37208558 58.0442445  71.02293086 74.21727929 74.23705148
 68.43198195 74.96700672 73.04465548 79.89595679 78.65834454 74.5570173
 64.75119234 59.77339198 57.61843562 60.14631826 59.94944502 73.71840951
 71.02293086 68.4555863  65.4596451  65.27633638 48.10793739 48.19759898]
[ 145.02380736  143.53789461  147.82567438  145.58562492  148.50350009
  147.81320596  151.48243977  149.48976259  155.1428032

In [ ]:
def var_key_retrieve(datapath, year, month):
    # get the path to each datasets
    print ("Start retrieving datasets {} (y) - {} (m)".format(year,namelist_month[month-1]))
    # The shape of each variable is (304,448)
    # Sea Ice Concentration
    datapath_key = glob.glob(os.path.join(datapath,"seaice_conc_monthly_nh_*_{0}{1}_v03r01.nc".format(year,namelist_month[month-1])))[0]
    # get the variable keys
    var_key = Dataset(datapath_key)

    print ("Retrieving datasets successfully and return the variable key!")
    return var_key

In [ ]:
def create_netcdf_point(pool_sicpm, output_path):
    print ('*******************************************************************')
    print ('*********************** create netcdf file*************************')
    print ('*******************************************************************')
    # wrap the datasets into netcdf file
    # 'NETCDF3_CLASSIC', 'NETCDF3_64BIT', 'NETCDF4_CLASSIC', and 'NETCDF4'
    data_wrap = Dataset(os.path.join(output_path, 'noaa_nsidc_monthly_regress_1979_2018_sic_passive_microwave.nc'),'w',format = 'NETCDF4')
    # create dimensions for netcdf data
    year_wrap_dim = data_wrap.createDimension('year',Dim_year)
    month_wrap_dim = data_wrap.createDimension('month',Dim_month)
    lat_wrap_dim = data_wrap.createDimension('latitude',Dim_latitude)
    lon_wrap_dim = data_wrap.createDimension('longitude',Dim_longitude)
    # create coordinate variables for 3-dimensions
    year_wrap_var = data_wrap.createVariable('year',np.int32,('year',))
    month_wrap_var = data_wrap.createVariable('month',np.int32,('month',))
    lat_wrap_var = data_wrap.createVariable('latitude',np.float32,('latitude',))
    lon_wrap_var = data_wrap.createVariable('longitude',np.float32,('longitude',))
    # create the actual 3-d variable
    sicpm_wrap_var = data_wrap.createVariable('sicpm',np.float32,('year','month','latitude','longitude'),zlib=True)
    # global attributes
    data_wrap.description = 'Monthly mean sea ice concentration with passive microwave by NOAA/NSIDC'
    # variable attributes
    lat_wrap_var.units = 'degree_north'
    lon_wrap_var.units = 'degree_east'

    sicpm_wrap_var.units = '1'

    sicpm_wrap_var.long_name = 'sea ice concentration with passive microwave'

    # writing data
    lat_wrap_var[:] = latitude
    lon_wrap_var[:] = longitude
    month_wrap_var[:] = index_month
    year_wrap_var[:] = period

    sicpm_wrap_var[:] = pool_sicpm

    # close the file
    data_wrap.close()
    print ("Create netcdf file successfully")